In [1]:
import pymysql
import pandas as pd
import numpy as np

# MySQL connection details
host = 'localhost'
user = 'root'
password = ''
database = 'Covid'

# File path to the CSV file
csv_file_path = ''

# Load the data into a pandas DataFrame
data = pd.read_csv(csv_file_path)

# Clean the data
data = data.replace({np.nan: None, np.inf: None, -np.inf: None})  # Replace NaN and invalid values

# Connect to MySQL
connection = pymysql.connect(host=host, user=user, password=password, database=database)
cursor = connection.cursor()

# Create the table schema based on actual columns
create_table_query = """
CREATE TABLE IF NOT EXISTS CovidVaccinations (
    iso_code VARCHAR(10),
    continent VARCHAR(50),
    location VARCHAR(255),
    date DATE,
    population BIGINT,
    total_cases FLOAT,
    new_cases FLOAT,
    new_cases_smoothed FLOAT,
    total_deaths FLOAT,
    new_deaths FLOAT,
    new_deaths_smoothed FLOAT,
    total_cases_per_million FLOAT,
    new_cases_per_million FLOAT,
    new_cases_smoothed_per_million FLOAT,
    total_deaths_per_million FLOAT,
    new_deaths_per_million FLOAT,
    new_deaths_smoothed_per_million FLOAT,
    reproduction_rate FLOAT,
    icu_patients FLOAT,
    icu_patients_per_million FLOAT,
    hosp_patients FLOAT,
    hosp_patients_per_million FLOAT,
    weekly_icu_admissions FLOAT,
    weekly_icu_admissions_per_million FLOAT,
    weekly_hosp_admissions FLOAT,
    weekly_hosp_admissions_per_million FLOAT
);
"""
cursor.execute(create_table_query)

# Insert data into the table
for _, row in data.iterrows():
    try:
        insert_query = """
        INSERT INTO CovidVaccinations (
            iso_code, continent, location, date, population, total_cases,
            new_cases, new_cases_smoothed, total_deaths, new_deaths,
            new_deaths_smoothed, total_cases_per_million, new_cases_per_million,
            new_cases_smoothed_per_million, total_deaths_per_million,
            new_deaths_per_million, new_deaths_smoothed_per_million,
            reproduction_rate, icu_patients, icu_patients_per_million,
            hosp_patients, hosp_patients_per_million, weekly_icu_admissions,
            weekly_icu_admissions_per_million, weekly_hosp_admissions,
            weekly_hosp_admissions_per_million
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        cursor.execute(insert_query, tuple(row))
    except pymysql.MySQLError as e:
        print(f"Skipping row due to MySQL error: {e}")
    except TypeError as e:
        print(f"Skipping row due to TypeError: {e}")

# Commit changes and close the connection
connection.commit()
cursor.close()
connection.close()
